In [ ]:
import os

Reference=input(str())  # input name of Genome Reference assembly until .gz
fastq=input(str())      # input name of fastq file without format file
Sample=input(str())     # input name output of sampel
vcf_ref=input(str())    # input name of vcf reference for annotation until .gz
min_ram=input(int())
max_ram=input(int())
thread=input(int())

In [ ]:
min_ram=input(int())
max_ram=input(int())
thread=input(int())

In [16]:
# Preparation of Reference
index_reference=("""bwa index """ +Reference)
Dictonary_ref=("""gatk --java-options "-Xms"""+min_ram+"""g -Xmx"""+max_ram+"""g" CreateSequenceDictionary -R """ +Reference)
Fastaidx_ref=("""samtools faidx """ +Reference)

In [2]:
# Alligentment fastq to reference
aligenment=("""bwa mem -t """ + thread + ' ' + Reference + ' '+fastq+"""_1.filt.fastq.gz """+ fastq+"""_2.filt.fastq.gz -o """+Sample+'.bam')

In [3]:
# Sorted sam/bam file
Sorted_Sam=("""gatk --java-options "-Xms"""+min_ram+"""g -Xmx"""+max_ram+"""g" SortSam -CREATE_INDEX true""" 
                """ -I """ +Sample+""".bam"""
                """ -O """ +Sample+""".sorted.bam"""
                """ -SO coordinate""")

# Mark and Deleted Duplicate
Mark_Duplicate=("""gatk --java-options "-Xms"""+min_ram+"""g -Xmx"""+max_ram+"""g" MarkDuplicates -CREATE_INDEX true""" 
                """ -I """ +Sample+""".sorted.bam"""
                """ -O """ +Sample+""".marked.bam"""
                """ -M """ +Sample+"""_Matric.txt --REMOVE_SEQUENCING_DUPLICATES true""")

# Replace null groups
Replace=("""gatk --java-options "-Xms"""+min_ram+"""g -Xmx"""+max_ram+"""g" AddOrReplaceReadGroups -CREATE_INDEX true""" 
                """ -I """ +Sample+""".marked.bam"""
                """ -O """ +Sample+""".replace.bam"""
                """ -RGID 1 -RGPL ILLUMINA -RGLB lib1 -RGPU unit1 -RGSM Sample""")

In [4]:
# Generate VCF
Haplotypecaller=("""gatk --java-options "-Xms"""+min_ram+"""g -Xmx"""+max_ram+"""g" HaplotypeCaller"""
                """ -R """ +Reference+
                """ -I """ +Sample+""".replace.bam"""
                """ -O """ +Sample+""".vcf"""
                """ --native-pair-hmm-threads """+ thread)

In [26]:
# Annotation
# Create index vcf
compress_sample=("""bgzip -@ """ + thread +' '+ Sample+'.vcf')
Sample_index=("bcftools index --force --output " +Sample+".vcf.gz.csi " +Sample+".vcf.gz")
Ref_index=("bcftools index --force --output " +vcf_ref+".vcf.gz.csi " +vcf_ref+".vcf.gz")

# rename chromosome
rename=("bcftools annotate --rename-chrs chr_names.txt " +Sample+'.vcf.gz -Oz -o '+ Sample+'_rename.vcf.gz')

# Annotate
Annotate=("""bcftools annotate -a """ +vcf_ref+""".vcf.gz -c ID -O z -o """+ Sample+"""_annotated.vcf.gz """+ Sample+"""_rename.vcf.gz --thread """ + thread)

# Fillter
Fillter=("""bcftools filter -e 'ID="."' """+Sample+"""_annotate.vcf.gz -o Trial_filtered.vcf""")


In [ ]:
#For running do os.system(fill bellow command)
#os.system(Reference)
#os.system(fastq)
#os.system(Sample)
#os.system(vcf_ref)
#os.system(index_reference)
#os.system(Dictonary_ref)
#os.system(Fastaidx_ref)
os.system(aligenment)
os.system(Sorted_Sam)
os.system(Mark_Duplicate)
os.system(Replace)
os.system(Haplotypecaller)
#os.system(compress_sample)
#os.system(Sample_index)
#os.system(Ref_index)
#os.system(rename)
#os.system(Annotate)
#os.system(Fillter)